In [2]:
import numpy as np
import numpy.linalg as la
import os
import copy
import warnings
import scipy.optimize as opt

# Load data from local files or from hard drive?

In [3]:
localData = False

# Define the model

In [4]:

nOrb = 3

nHole = 0

tmd = 'MoS_2'


if tmd == 'MoS_2' :

    abs_t0 = 0.184

    e1 = 1.046 / abs_t0
    e2 = 2.104 / abs_t0
    t0 = - 1
    t1 = 0.401 / abs_t0
    t2 = 0.507 / abs_t0
    t11 = 0.218 / abs_t0
    t12 = 0.338 / abs_t0
    t22 = 0.057 / abs_t0

E0 = np.array([[e1, 0, 0],
               [0, e2, 0],
               [0, 0, e2]])

E1 = np.array([[t0, t1, t2],
               [-t1, t11, t12],
               [t2, -t12, t22]])

E4 = np.array([[t0, -t1, t2],
               [t1, t11, -t12],
               [t2, t12, t22]])

E2 = np.array([[t0, 0.5 * t1 - np.sqrt(3) / 2 * t2, - np.sqrt(3) / 2 * t1 - 0.5 * t2],
               [-0.5 * t1 - np.sqrt(3) / 2 * t2, 0.25 * ( t11 + 3 * t22 ), np.sqrt(3) / 4 * ( t22 - t11 ) - t12],
               [np.sqrt(3) / 2 * t1 - 0.5 * t2, np.sqrt(3) / 4 * ( t22 - t11 ) + t12, ( 3 * t11 + t22) / 4 ]])

E5 = np.array([[t0, - 0.5 * t1 - np.sqrt(3) / 2 * t2, np.sqrt(3) / 2 * t1 - 0.5 * t2],
               [0.5 * t1 - np.sqrt(3) / 2 * t2, 0.25 * ( t11 + 3 * t22 ), np.sqrt(3) / 4 * ( t22 - t11 ) + t12],
               [-np.sqrt(3) / 2 * t1 - 0.5 * t2, np.sqrt(3) / 4 * ( t22 - t11 ) - t12, ( 3 * t11 + t22) / 4 ]])

E3 = np.array([[t0, - 0.5 * t1 + np.sqrt(3) / 2 * t2, -np.sqrt(3) / 2 * t1 - 0.5 * t2],
               [0.5 * t1 + np.sqrt(3) / 2 * t2, 0.25 * ( t11 + 3 * t22 ), -np.sqrt(3) / 4 * ( t22 - t11 ) + t12],
               [np.sqrt(3) / 2 * t1 - 0.5 * t2, -np.sqrt(3) / 4 * ( t22 - t11 ) - t12, ( 3 * t11 + t22) / 4 ]])

E6 = np.array([[t0, 0.5 * t1 + np.sqrt(3) / 2 * t2, np.sqrt(3) / 2 * t1 - 0.5 * t2],
               [-0.5 * t1 + np.sqrt(3) / 2 * t2, 0.25 * ( t11 + 3 * t22 ), -np.sqrt(3) / 4 * ( t22 - t11 ) - t12],
               [-np.sqrt(3) / 2 * t1 - 0.5 * t2, -np.sqrt(3) / 4 * ( t22 - t11 ) + t12, ( 3 * t11 + t22) / 4 ]])

hoppings = np.array([E0, E1, E2, E3, E4, E5, E6])

inftyCutOff = 1000 # above it, beta is practically infinity

def fermi(e, mu, beta):
    '''
        For zero temperature, set beta = inftyCutOff
        '''
    if beta == inftyCutOff:
        return (e < mu).astype(int)
    else:
        return 1 / ( 1 + np.exp( beta * ( e - mu ) ) )

def iTriang(x, y, Nx, Ny):
    return Nx * y + x

def triangularNano(Nx, Ny, nOrb, hoppings):
    T = np.zeros((nOrb*Nx*Ny, nOrb*Nx*Ny))
    for x in range(Nx):
        for y in range(Ny):
            # Diagonal term
            T[ iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny) + 1) * nOrb,\
              iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny) + 1) * nOrb ]\
            = hoppings[0]
            
            # E1
            T[ iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny) + 1)*nOrb,\
              iTriang( (x + 1) % Nx , y, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , y, Nx, Ny) + 1)*nOrb ]\
              = hoppings[1]
            
            # E4
            T[ iTriang( (x + 1) % Nx , y, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , y, Nx, Ny) + 1)*nOrb\
              , iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb ] = hoppings[4]
            
            if y == 0:
                T[ iTriang(x, 0, Nx, Ny)*nOrb:(iTriang(x, 0, Nx, Ny)+1)*nOrb,\
                  iTriang( x, 1, Nx, Ny)*nOrb:(iTriang( x, 1, Nx, Ny)+1)*nOrb ]\
                  = hoppings[6]
                
                T[ iTriang(x, 1, Nx, Ny)*nOrb:(iTriang(x, 1, Nx, Ny)+1)*nOrb,\
                  iTriang( x, 0, Nx, Ny)*nOrb:(iTriang( x, 0, Nx, Ny)+1)*nOrb ]\
                  = hoppings[3]
                
                if x == 0:
                    T[iTriang(x, 0, Nx, Ny)*nOrb:(iTriang(x, 0, Nx, Ny) + 1)*nOrb,\
                      iTriang( Nx - 1, 1, Nx, Ny)*nOrb:(iTriang( Nx - 1, 1, Nx, Ny) + 1)*nOrb]\
                      = hoppings[5]
                    T[iTriang( Nx - 1, 1, Nx, Ny)*nOrb:(iTriang( Nx - 1, 1, Nx, Ny)+1)*nOrb,\
                      iTriang(x, 0, Nx, Ny)*nOrb:(iTriang(x, 0, Nx, Ny)+1)*nOrb] = hoppings[2]
                else:
                    T[iTriang(x, 0, Nx, Ny)*nOrb:(iTriang(x, 0, Nx, Ny)+1)*nOrb,\
                      iTriang( x - 1, 1, Nx, Ny)*nOrb:(iTriang( x - 1, 1, Nx, Ny)+1)*nOrb] = hoppings[5]
                    T[iTriang(x - 1, 1, Nx, Ny)*nOrb:(iTriang(x - 1, 1, Nx, Ny)+1)*nOrb,\
                      iTriang(x, 0, Nx, Ny)*nOrb:(iTriang(x, 0, Nx, Ny)+1)*nOrb] = hoppings[2]
        else:
            if y == Ny - 1:
                T[iTriang(x, Ny - 1 , Nx, Ny)*nOrb:(iTriang(x, Ny - 1 , Nx, Ny) + 1)*nOrb,\
                  iTriang( (x + 1) % Nx , Ny - 2, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , Ny - 2, Nx, Ny) + 1)*nOrb]= hoppings[2]
                T[iTriang( (x + 1) % Nx , Ny - 2, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , Ny - 2, Nx, Ny)+1)*nOrb,\
                  iTriang(x, Ny - 1, Nx, Ny)*nOrb:(iTriang(x, Ny - 1, Nx, Ny)+1)*nOrb] = hoppings[5]
                T[iTriang(x, Ny - 1, Nx, Ny)*nOrb:(iTriang(x, Ny - 1, Nx, Ny)+1)*nOrb,\
                  iTriang( x, Ny - 2, Nx, Ny)*nOrb:(iTriang( x, Ny - 2, Nx, Ny)+1)*nOrb] = hoppings[3]
                T[iTriang(x, Ny - 2, Nx, Ny)*nOrb:(iTriang(x, Ny - 2, Nx, Ny)+1)*nOrb,\
                  iTriang( x, Ny - 1, Nx, Ny)*nOrb:(iTriang( x, Ny - 1, Nx, Ny)+1)*nOrb] = hoppings[6]
                
            else:
                T[iTriang(x, y , Nx, Ny)*nOrb:(iTriang(x, y , Nx, Ny)+1)*nOrb,\
                  iTriang( (x + 1) % Nx , y - 1, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , y - 1, Nx, Ny)+1)*nOrb] = hoppings[2]
                T[iTriang( (x + 1) % Nx , y - 1, Nx, Ny)*nOrb:(iTriang( (x + 1) % Nx , y - 1, Nx, Ny)+1)*nOrb,\
                  iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb] = hoppings[5]
                T[iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb,\
                  iTriang( x, y - 1, Nx, Ny)*nOrb:(iTriang( x, y - 1, Nx, Ny)+1)*nOrb] = hoppings[3]
                T[iTriang(x, y - 1, Nx, Ny)*nOrb:(iTriang(x, y - 1, Nx, Ny)+1)*nOrb,\
                  iTriang( x, y, Nx, Ny)*nOrb:(iTriang( x, y, Nx, Ny)+1)*nOrb] = hoppings[6]
                if x == 0:
                    T[iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb,\
                      iTriang( Nx - 1, y + 1, Nx, Ny)*nOrb:(iTriang( Nx - 1, y + 1, Nx, Ny)+1)*nOrb] = hoppings[5]
                    T[iTriang( Nx - 1, y + 1, Nx, Ny)*nOrb:(iTriang( Nx - 1, y + 1, Nx, Ny)+1)*nOrb,\
                      iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb] = hoppings[2]
                else:
                    T[iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb,\
                      iTriang(x - 1, y + 1, Nx, Ny)*nOrb:(iTriang(x - 1, y + 1, Nx, Ny)+1)*nOrb] = hoppings[5]
                    T[iTriang(x - 1, y + 1, Nx, Ny)*nOrb:(iTriang(x - 1, y + 1, Nx, Ny)+1)*nOrb,\
                      iTriang(x, y, Nx, Ny)*nOrb:(iTriang(x, y, Nx, Ny)+1)*nOrb] = hoppings[2]
    return T

def Hribbon(k, Ny):
    Hrib = np.zeros((3 * Ny, 3 * Ny), dtype=np.complex64)
    
    h1 = np.array([[e1 + 2 * t0 * np.cos(k),
                2.j * np.sin(k) * t1,
                2 * t2 * np.cos(k)],
               
               [-2.j * np.sin(k) * t1,
                e2 + 2 * t11 * np.cos(k),
                2.j * np.sin(k) * t12],
               
               [2 * t2 * np.cos(k),
                -2.j * np.sin(k) * t12,
                e2 + 2 * t22 * np.cos(k)]
               
               ], dtype=np.complex64)
        
    h2 = np.array([
              
          [ 2 * t0 * np.cos(k/2) ,
           1.j * np.sin(k/2) * ( t1 - np.sqrt(3) * t2 ) ,
           -1. * np.cos(k/2) * ( np.sqrt(3) * t1 + t2 )] ,
          
          [ -1.j * np.sin(k/2) * ( t1 + np.sqrt(3) * t2 ),
           0.5 * np.cos(k/2) * ( t11 + 3 * t22 ),
           1.j * np.sin(k/2) * ( np.sqrt(3) / 2 * ( t22 - t11 ) - 2 * t12 ) ],
          
          [ np.cos(k/2) * ( np.sqrt(3) * t1 - t2 ),
           1.j * np.sin(k/2) * ( np.sqrt(3)/2 * ( t22 - t11 ) + 2 * t12 ),
           0.5 * np.cos(k/2) * ( 3 * t11 + t22 ) ]
          
          ], dtype=np.complex64)
   
    for y in range(1, Ny-1):
       Hrib[3*y:3*(y+1), 3*y:3*(y+1)] = h1
       Hrib[3*(y-1):3*y, 3*y:3*(y+1)] = (h2.conj()).T
       Hrib[3*(y+1):3*(y+2), 3*y:3*(y+1)] = h2

    Hrib[0:3, 0:3] = h1
    Hrib[3*(Ny-1):3*(Ny), 3*(Ny-1):3*(Ny)] = h1
    Hrib[3*(Ny-2):3*(Ny-1), 3*(Ny-1):3*Ny] = (h2.conj()).T
    Hrib[3:6, 0:3] = h2

    return Hrib


def solve_self_consistent(Nx, Ny, invTemp, U, initCond):
    '''
        Nx : Number of ks;
        Ny : Number of atoms along the ribbon's transverse direction
        intTemp : Inverse Temperature Beta
        U : On-site interaction
        initCond : 1: Ferromagnetic, 2: AF, 3: Paramagnetic
        
        Returns nUp, nDown, energies, itSwitch, lastIt, eUp, eDown, wUp, wDown
    '''
    
    K = triangularNano(Nx, Ny, nOrb, hoppings)
    
    N = nOrb * Nx * Ny # number of sites (orbital + spatial)

    beta0 = 1.5 # must be > 1, otherwise beta decreases
    betaTarget = invTemp # inftyCutOff in fermi means infty , i.e. T = 0
    beta = beta0 # beta starts as beta0

    itMax = 100
    it = 0
    lbda = 0.5 / (1.1 * itMax) # the factor multiplied by itMax impedes P ( I ) < \delta
    itSwitch = 0
    dampFreq = 1

    np.random.seed(1)

    if initCond == 1:
        
        nUp = np.ones(3*Ny) - 0.01 * np.random.rand(3*Ny)
        nDown = np.zeros(3*Ny) + 0.01 * np.random.rand(3*Ny)
    
    if initCond == 2:
        
        nUp = np.zeros(3*Ny)
        nDown = np.zeros(3*Ny)

        spinFlipper = 0.5

        for i in range(3*Ny):
            if i % (3) == 0:
                spinFlipper *= -1
            nUp[i] = 0.5 + spinFlipper *.1 * np.random.rand()
            nDown[i] = 0.5 - spinFlipper *.1 * np.random.rand()

    if initCond == 3:
        nUp = np.ones(3*Ny) - 0.01 * np.random.rand(3*Ny)

        nDown = np.ones(3*Ny) + 0.01 * np.random.rand(3*Ny)

    # Initialize energies
    energies = np.zeros(itMax)

    # Tolerance

    delta = 1e-10

    deltaUp = delta + 1
    deltaDown = delta + 1

    eUp = np.zeros((Nx, 3*Ny))
    wUp = np.zeros((Nx, 3*Ny, 3*Ny), dtype=np.complex64)
    eDown = np.zeros((Nx, 3*Ny))
    wDown = np.zeros((Nx, 3*Ny, 3*Ny), dtype=np.complex64)

    ks = np.linspace(-np.pi,np.pi, num=Nx, endpoint=False)

    while (it < itMax and deltaUp > delta and deltaDown > delta):
        
       # Annealing
       
        if (beta < inftyCutOff \
           and beta < betaTarget) : # > infty: zero temperature case
           beta = beta0 ** it
           if beta > betaTarget:
               itSwitch = it
               print(itSwitch)
               beta = betaTarget
        else:
            beta = betaTarget

        print('beta: ', beta)

        for kCount, k in enumerate(ks):

            C = - nUp * nDown
    
            K = Hribbon(k, Ny)
    
            Hup = K + U * np.eye(3*Ny) * ( nDown + C / 2 )
            Hdown = K + U * np.eye(3*Ny) * ( nUp + C / 2 )
        
            eUp[kCount, :], wUp[kCount, :, :] = la.eigh(Hup)
            eDown[kCount, :], wDown[kCount, :, :] = la.eigh(Hdown)
    
        nUpOld = nUp.copy()
        nDownOld = nDown.copy()

        def rootToChem(chemPot):
            return ( np.sum(fermi(eUp, chemPot, beta )) \
                    +  np.sum(fermi(eDown, chemPot, beta )) ) / Nx / Ny - (2 - nHole)

        mu = opt.bisect(rootToChem, -50, 50)
    
        for i in range(3*Ny):
            nUp[i] = 0
            nDown[i] = 0
            for n in range(3*Ny):
                for q in range(Nx):
                    nUp[i] += abs(wUp[q, i, n])**2 * fermi(eUp[q, n].real, mu , beta)
                    nDown[i] += abs(wDown[q, i, n])**2 * fermi(eDown[q, n].real, mu, beta)
            nUp[i] /= Nx
            nDown[i] /= Nx

        # Damping
        if it % dampFreq == 0:
            nUp = ( 1 / 2 + lbda * it ) * nUp\
            + ( 1 / 2 - lbda * it) * nUpOld
            nDown = ( 1 / 2 + lbda * it ) * nDown\
            + ( 1 / 2 - lbda * it) * nDownOld
    
        deltaUp = np.dot(nUp - nUpOld, nUp - nUpOld) / np.dot(nUpOld, nUpOld)
        deltaDown = np.dot(nDown - nDownOld, nDown - nDownOld) / np.dot(nDownOld, nDownOld)

        # To check convergence
        print('delta nUp: ', deltaUp)
        print('delta nDown: ', deltaDown)
        # Check if chemical potential is imposing
        # the right number of particles
        print('<n>: ', (nUp.sum() + nDown.sum() ) / ( 3 * Ny ) )
        
        if beta < inftyCutOff:
            energies[it] = U * np.dot(nUp, nDown) + mu * (nUp + nDown).sum() \
            - 1 / beta * ( ( np.log( 1 + np.exp( - beta * ( eUp - mu ) ) ) ).sum() + \
                          (np.log( 1 + np.exp( - beta * ( eDown - mu ) ) ) ).sum() )
        else:
            energies[it] = U * np.dot(nUp, nDown) + mu * (nUp + nDown).sum()
            
        it += 1
            
    lastIt = it
    print(lastIt)

    return nUp, nDown, energies, itSwitch, lastIt, eUp, eDown, mu,\
np.absolute(wUp.flatten('C'))**2, np.absolute(wDown.flatten('C'))**2

def savedata(SAVESUBDIR, data):
    np.savetxt(SAVESUBDIR + "nUp.txt", data[0])
    np.savetxt(SAVESUBDIR + "nDown.txt", data[1])
    np.savetxt(SAVESUBDIR + "energies.txt", data[2])
    np.savetxt(SAVESUBDIR + "itSwitch_lastIt_mu.txt", (data[3], data[4], data[7]))
    np.savetxt(SAVESUBDIR + "eUp.txt", (data[5]))
    np.savetxt(SAVESUBDIR + "eDown.txt", (data[6]))
    np.savetxt(SAVESUBDIR + "wUp.txt", (data[8]))
    np.savetxt(SAVESUBDIR + "wDown.txt", (data[9]))
    np.savetxt(SAVESUBDIR + "modelParams.txt",\
              (abs_t0, e1, e2, t0, t1, t2, t11, t12, t22))

In [5]:
cwd = os.getcwd()

if localData == False:
    
    ## SAVE FILES TO HARD DRIVE (THEY ARE TOO BIG!). SET PATH BELOW.
    SAVEDIR = "../../../../../../../Volumes/ADATA HD710/MScData/MeanFieldTMDnanoribbon/"
else:
    ## SAVE FILES IN plots FOLDER
    SAVEDIR = "../plots/MeanFieldTMDnanoribbon/"

## Sweep U parameter for T = 0 (zero temperature)

In [6]:
Nx = 512
Ny = 16
beta = 100
initCond = 1

#Us = [1, 3, 5, 7, 10, 15, 20]

#Us = np.arange(18, 20)

Us = np.arange(14.8, 16.2, 0.2)

for U in Us:
    SAVESUBDIR = SAVEDIR + "/Nx=" + str(Nx) +\
        "_Ny=" + str(Ny) + "_U=" + str(U) + "_beta=" + str(beta) + "/"
    if not os.path.exists(SAVESUBDIR):
        os.makedirs(SAVESUBDIR)

    data = solve_self_consistent(Nx, Ny, beta, U, initCond)
    savedata(SAVESUBDIR, data)

beta:  1.0
delta nUp:  0.0714532428001
delta nDown:  304.467370307
<n>:  0.833398829818
beta:  1.5
delta nUp:  0.0667881328492
delta nDown:  1.15087486123
<n>:  0.749274874756
beta:  2.25
delta nUp:  0.0345197209092
delta nDown:  0.0924469526266
<n>:  0.707219787007
beta:  3.375
delta nUp:  0.0133245658994
delta nDown:  0.0165929587971
<n>:  0.686390229792
beta:  5.0625
delta nUp:  0.00429727363534
delta nDown:  0.00388513838037
<n>:  0.676169837958
beta:  7.59375
delta nUp:  0.00127548832167
delta nDown:  0.00105070707924
<n>:  0.671202271197
beta:  11.390625


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: overflow encountered in exp


delta nUp:  0.00037979804595
delta nDown:  0.000327136245596
<n>:  0.668810770571
beta:  17.0859375
delta nUp:  0.000125444008631
delta nDown:  0.00011987970045
<n>:  0.667670497022
beta:  25.62890625
delta nUp:  5.01753593211e-05
delta nDown:  5.16590353107e-05
<n>:  0.667132078899
beta:  38.443359375
delta nUp:  2.46633829298e-05
delta nDown:  2.5558716523e-05
<n>:  0.666880333222
beta:  57.6650390625
delta nUp:  1.41036996126e-05
delta nDown:  1.43050342463e-05
<n>:  0.666763787794
beta:  86.49755859375
delta nUp:  8.92342437562e-06
delta nDown:  9.29456273569e-06
<n>:  0.66671037116
12
beta:  100


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:284: RuntimeWarning: overflow encountered in exp


delta nUp:  6.39501937441e-06
delta nDown:  7.59910897878e-06
<n>:  0.666686135116
beta:  100
delta nUp:  4.86777475757e-06
delta nDown:  7.16180510601e-06
<n>:  0.666675250462
beta:  100
delta nUp:  4.2657150043e-06
delta nDown:  7.47068052833e-06
<n>:  0.666670412261
beta:  100
delta nUp:  4.093891731e-06
delta nDown:  8.44492181259e-06
<n>:  0.666668284015
beta:  100
delta nUp:  4.17866141121e-06
delta nDown:  9.30951394528e-06
<n>:  0.666667357537
beta:  100
delta nUp:  4.19252403675e-06
delta nDown:  9.65812871194e-06
<n>:  0.666666958724
beta:  100
delta nUp:  4.62353030572e-06
delta nDown:  1.03696646653e-05
<n>:  0.6666667888
beta:  100
delta nUp:  4.51843143128e-06
delta nDown:  1.05866618443e-05
<n>:  0.666666717183
beta:  100
delta nUp:  4.63128230476e-06
delta nDown:  1.03676193205e-05
<n>:  0.666666687343
beta:  100
delta nUp:  4.79272880971e-06
delta nDown:  1.00959678301e-05
<n>:  0.666666675165
beta:  100
delta nUp:  4.41592190324e-06
delta nDown:  9.59831310347e-06
<n>

delta nUp:  2.55218043317e-05
delta nDown:  2.83641124509e-05
<n>:  0.666880333311
beta:  57.6650390625
delta nUp:  1.42246764791e-05
delta nDown:  1.54240769097e-05
<n>:  0.666763787849
beta:  86.49755859375
delta nUp:  8.53881130847e-06
delta nDown:  8.86492724837e-06
<n>:  0.666710371269
12
beta:  100
delta nUp:  5.48192334781e-06
delta nDown:  5.54487142178e-06
<n>:  0.666686135182
beta:  100
delta nUp:  3.59867970616e-06
delta nDown:  3.77933194489e-06
<n>:  0.666675250481
beta:  100
delta nUp:  2.47317123735e-06
delta nDown:  2.9244628721e-06
<n>:  0.666670412457
beta:  100
delta nUp:  1.91836072051e-06
delta nDown:  2.84981712892e-06
<n>:  0.666668284139
beta:  100
delta nUp:  1.74545614449e-06
delta nDown:  3.06836375496e-06
<n>:  0.666667357661
beta:  100
delta nUp:  1.73545419532e-06
delta nDown:  3.80989195579e-06
<n>:  0.666666958768
beta:  100
delta nUp:  1.8995829746e-06
delta nDown:  4.44104669192e-06
<n>:  0.666666788843
beta:  100
delta nUp:  2.21212151486e-06
delta nD

delta nUp:  1.25973304061e-10
delta nDown:  1.28380037377e-10
<n>:  0.666666666653
beta:  100
delta nUp:  9.51365030774e-11
delta nDown:  9.7287223157e-11
<n>:  0.666666666531
99
beta:  1.0
delta nUp:  0.0680979061861
delta nDown:  264.396832799
<n>:  0.833398829836
beta:  1.5
delta nUp:  0.0664046804196
delta nDown:  1.32747569077
<n>:  0.749274874737
beta:  2.25
delta nUp:  0.035345207463
delta nDown:  0.104908603487
<n>:  0.707219787031
beta:  3.375
delta nUp:  0.0138889818447
delta nDown:  0.0185859688002
<n>:  0.68639022983
beta:  5.0625
delta nUp:  0.00451355246662
delta nDown:  0.0043339328582
<n>:  0.676169838042
beta:  7.59375
delta nUp:  0.0013515168788
delta nDown:  0.00117908290746
<n>:  0.671202271124
beta:  11.390625
delta nUp:  0.000407031923506
delta nDown:  0.000371193485414
<n>:  0.668810770599
beta:  17.0859375
delta nUp:  0.000136503036885
delta nDown:  0.000139130737603
<n>:  0.667670497165
beta:  25.62890625
delta nUp:  5.52940068822e-05
delta nDown:  6.1951316392

delta nUp:  1.71781988056e-08
delta nDown:  1.93641515438e-08
<n>:  0.666666666768
beta:  100
delta nUp:  1.44723746329e-08
delta nDown:  1.66878645228e-08
<n>:  0.666666666624
beta:  100
delta nUp:  1.18146903214e-08
delta nDown:  1.3681578988e-08
<n>:  0.666666666663
beta:  100
delta nUp:  9.48110536571e-09
delta nDown:  1.08324260294e-08
<n>:  0.666666666804
beta:  100
delta nUp:  7.55099728621e-09
delta nDown:  8.41878782436e-09
<n>:  0.666666666738
beta:  100
delta nUp:  5.99450144831e-09
delta nDown:  6.49979073484e-09
<n>:  0.666666666872
beta:  100
delta nUp:  4.7539461057e-09
delta nDown:  5.02070894692e-09
<n>:  0.666666666768
beta:  100
delta nUp:  3.77228568098e-09
delta nDown:  3.89583342362e-09
<n>:  0.666666666705
beta:  100
delta nUp:  2.99998012871e-09
delta nDown:  3.04294824622e-09
<n>:  0.666666666698
beta:  100
delta nUp:  2.39456023024e-09
delta nDown:  2.39553697714e-09
<n>:  0.666666666665
beta:  100
delta nUp:  1.92114290448e-09
delta nDown:  1.90186595082e-09


delta nUp:  0.0367729782131
delta nDown:  0.13131712735
<n>:  0.707219787015
beta:  3.375
delta nUp:  0.0148769410239
delta nDown:  0.0223703156037
<n>:  0.686390229684
beta:  5.0625
delta nUp:  0.00489102783596
delta nDown:  0.0051589148997
<n>:  0.676169837955
beta:  7.59375
delta nUp:  0.0014859506267
delta nDown:  0.00141731619772
<n>:  0.671202271121
beta:  11.390625
delta nUp:  0.000456339006361
delta nDown:  0.000451025245869
<n>:  0.668810770604
beta:  17.0859375
delta nUp:  0.000157946568946
delta nDown:  0.000172604522875
<n>:  0.667670497138
beta:  25.62890625
delta nUp:  6.68066426224e-05
delta nDown:  7.90584145185e-05
<n>:  0.667132079015
beta:  38.443359375
delta nUp:  3.4308111939e-05
delta nDown:  4.15084424805e-05
<n>:  0.666880333186
beta:  57.6650390625
delta nUp:  1.99228478068e-05
delta nDown:  2.37029087431e-05
<n>:  0.666763787902
beta:  86.49755859375
delta nUp:  1.22576004486e-05
delta nDown:  1.41536782336e-05
<n>:  0.666710371168
12
beta:  100
delta nUp:  7.

KeyboardInterrupt: 